In [1]:
!rm -rf MQRNN/
!git clone https://github.com/tianchen101/MQRNN.git

In [2]:
# import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# from MQRNN import Encoder, Decoder
# from MQRNN.MQRNN import MQRNN
# from MQRNN.data import MQRNN_dataset, read_df

In [4]:
config = {
    'horizon_size': 40,
    'hidden_size': 50,
    'quantiles': [0.1, 0.5, 0.8], 
    'columns': [1],
    'dropout': 0.3,
    'layer_size': 2,
    'by_direction': False,
    'lr': 1e-3,
    'batch_size': 1,
    'num_epochs': 500,
    'context_size': 10,
    'train_ratio': 0.8,
}

# Thiết lập device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

NameError: name 'torch' is not defined

In [5]:
train = pd.read_csv('./data/rossmann-store-sales/train.csv', low_memory=False)
test = pd.read_csv('./data/rossmann-store-sales/test.csv', low_memory=False)
store = pd.read_csv('./data/rossmann-store-sales/store.csv', low_memory=False)
print("Successfully loaded datasets.")

Successfully loaded datasets.


In [6]:
test.fillna(1,inplace=True)

store.CompetitionDistance = store.CompetitionDistance.fillna(store.CompetitionDistance.median())
store.fillna(0,inplace=True)
store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,0.0,0.0,0
4,5,a,a,29910.0,4.0,2015.0,0,0.0,0.0,0


In [7]:
train = pd.merge(train, store, on='Store')
test = pd.merge(test, store, on='Store')

In [8]:
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,9.0,2009.0,0,0.0,0.0,0
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.0,4.0,2015.0,0,0.0,0.0,0


In [9]:
# Convert Date to datetime
train['Date'] = pd.to_datetime(train['Date'])
test['Date'] = pd.to_datetime(test['Date'])

# Create time features
for df in [train, test]:
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Day'] = df['Date'].dt.day
    df['DayOfWeek'] = df['Date'].dt.dayofweek
    df['WeekOfYear'] = df['Date'].dt.isocalendar().week.astype(int)

# Normalize numeric features
numeric_features = ['CompetitionDistance', 'CompetitionOpenSinceMonth', 
                   'CompetitionOpenSinceYear', 'Promo2SinceWeek', 'Promo2SinceYear']

for feature in numeric_features:
    mean = train[feature].mean()
    std = train[feature].std()
    train[feature] = (train[feature] - mean) / std
    test[feature] = (test[feature] - mean) / std

In [11]:
# Normalize Sales per store
train['Sales'] = train.groupby('Store')['Sales'].transform(
    lambda x: (x - x.mean()) / x.std()
)

# Create target and covariate dataframes
def create_mqrnn_dataset(df, target_col='Sales', covariate_cols=None):
    if covariate_cols is None:
        covariate_cols = ['Year', 'Month', 'Day', 'DayOfWeek', 'WeekOfYear',
                         'CompetitionDistance', 'CompetitionOpenSinceMonth',
                         'CompetitionOpenSinceYear', 'Promo2SinceWeek', 'Promo2SinceYear',
                         'Promo', 'StateHoliday', 'SchoolHoliday', 'Open']
    
    df = df.sort_values(['Store', 'Date'])
    target_series = df.pivot(index='Date', columns='Store', values=target_col)
    covariate_df = df.pivot(index='Date', columns='Store', values=covariate_cols)
    covariate_df.columns = [f"{col}_{store}" for col, store in covariate_df.columns]
    return target_series, covariate_df

# Create datasets
train_target, train_covariates = create_mqrnn_dataset(train)


In [12]:
# Create MQRNN dataset
class MQRNN_Dataset(torch.utils.data.Dataset):
    def __init__(self, X, y, covariates):
        self.X = X
        self.y = y
        self.covariates = covariates

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        input_window = self.X.shape[1]
        input_series = self.X[idx]
        input_covariate = self.covariates[idx, :input_window, :]
        output_window = self.y.shape[1]
        future_covariate = self.covariates[idx, input_window:, :]
        target = self.y[idx]

        input_series = torch.tensor(input_series, dtype=torch.float32).unsqueeze(-1)
        input_covariate = torch.tensor(input_covariate, dtype=torch.float32)
        future_covariate = torch.tensor(future_covariate, dtype=torch.float32)
        target = torch.tensor(target, dtype=torch.float32)

        encoder_input = torch.cat([input_series, input_covariate], dim=1)
        return encoder_input, future_covariate, target

# Create full dataset and split
full_dataset = MQRNN_Dataset(
    X=train_target.values,
    y=train_target.values,
    covariates=train_covariates.values
)

train_size = int(config['train_ratio'] * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])


NameError: name 'torch' is not defined

In [ ]:
# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config['batch_size'])

In [ ]:
def train_epoch(model, train_loader, optimizer, device):
    model.train()
    total_loss = 0
    for encoder_input, future_covariate, target in train_loader:
        encoder_input = encoder_input.to(device)
        future_covariate = future_covariate.to(device)
        target = target.to(device)

        optimizer.zero_grad()
        loss = model(encoder_input, future_covariate, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

def validate(model, val_loader, device):
    model.eval()
    total_loss = 0
    all_predictions = []
    all_targets = []
    
    with torch.no_grad():
        for encoder_input, future_covariate, target in val_loader:
            encoder_input = encoder_input.to(device)
            future_covariate = future_covariate.to(device)
            target = target.to(device)
            
            loss = model(encoder_input, future_covariate, target)
            total_loss += loss.item()
            
            predictions = model.predict(encoder_input, future_covariate)
            all_predictions.extend(predictions.cpu().numpy())
            all_targets.extend(target.cpu().numpy())
    
    all_predictions = np.array(all_predictions)
    all_targets = np.array(all_targets)
    
    mae = mean_absolute_error(all_targets, all_predictions)
    mse = mean_squared_error(all_targets, all_predictions)
    rmse = np.sqrt(mse)
    r2 = r2_score(all_targets, all_predictions)
    
    return total_loss / len(val_loader), mae, mse, rmse, r2

In [ ]:
# Initialize model
model = MQRNN(
    input_size=1 + len(get_feature_names()),
    hidden_size=config['hidden_size'],
    num_quantiles=len(config['quantiles']),
    dropout=config['dropout'],
    layer_size=config['layer_size']
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])

# Training loop
best_val_loss = float('inf')
train_losses = []
val_losses = []
val_metrics = []

for epoch in range(config['num_epochs']):
    train_loss = train_epoch(model, train_loader, optimizer, device)
    val_loss, mae, mse, rmse, r2 = validate(model, val_loader, device)
    
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    val_metrics.append([mae, mse, rmse, r2])
    
    print(f'Epoch {epoch+1}/{config["num_epochs"]}:')
    print(f'Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
    print(f'MAE: {mae:.4f}, MSE: {mse:.4f}, RMSE: {rmse:.4f}, R2: {r2:.4f}')
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': train_loss,
            'val_loss': val_loss,
            'val_metrics': [mae, mse, rmse, r2]
        }, 'best_model.pth')

In [ ]:
# Plot training and validation losses
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Losses')
plt.legend()
plt.show()

In [ ]:
# Plot metrics
metrics = np.array(val_metrics)
plt.figure(figsize=(15, 5))
plt.subplot(1, 4, 1)
plt.plot(metrics[:, 0], label='MAE')
plt.title('MAE')
plt.subplot(1, 4, 2)
plt.plot(metrics[:, 1], label='MSE')
plt.title('MSE')
plt.subplot(1, 4, 3)
plt.plot(metrics[:, 2], label='RMSE')
plt.title('RMSE')
plt.subplot(1, 4, 4)
plt.plot(metrics[:, 3], label='R2')
plt.title('R2')
plt.tight_layout()
plt.show()